In [1]:
# RESUPPLY

# sales
# https://caenergy-my.sharepoint.com/:x:/g/personal/jesten_ruiz_energy_ca_gov/ETcZua8EH95EkQEOA0a3OZcBFqAQlmGkfitkEA3KnFapWg?email=Fernando.Greve%40Energy.ca.gov&wdOrigin=TEAMS-MAGLEV.p2p_ns.rwc&wdExp=TEAMS-TREATMENT&wdhostclicktime=1736878801692&web=1&clickparams=eyJBcHBOYW1lIjoiVGVhbXMtRGVza3RvcCIsIkFwcFZlcnNpb24iOiI0OS8yNDEyMDEwMDIxMyIsIkhhc0ZlZGVyYXRlZFVzZXIiOmZhbHNlfQ%3D%3D
# https://caenergy-my.sharepoint.com/:x:/g/personal/jesten_ruiz_energy_ca_gov/EZ5jsGmt02BDqpl4Vsupu4gBiKP1g9FcLwppVhztSZ98qg?email=Fernando.Greve%40Energy.ca.gov&wdOrigin=TEAMS-MAGLEV.p2p_ns.rwc&wdExp=TEAMS-TREATMENT&wdhostclicktime=1736878863838&web=1&clickparams=eyJBcHBOYW1lIjoiVGVhbXMtRGVza3RvcCIsIkFwcFZlcnNpb24iOiI0OS8yNDEyMDEwMDIxMyIsIkhhc0ZlZGVyYXRlZFVzZXIiOmZhbHNlfQ%3D%3D


In [2]:
import pyodbc
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [3]:
wd = os.getcwd()
rawdata_path = os.path.join(wd, "rawdata")
data_path = os.path.join(wd, "data")

In [4]:
def df_to_rawdata(df, filename):
    """Save df into rawdata folder."""    
    # df['Date'] = df.index
    # df = df.set_index("Date")    
    df.to_csv(os.path.join(rawdata_path, filename + ".csv"), index=False)

In [5]:
def rawdata_to_df(filename):
    # df['Date'] = df.index
    # df = df.set_index("Date")
    df = pd.read_csv(os.path.join(rawdata_path, filename + ".csv"))    
    return df

In [6]:
def df_to_data(df, filename):
    """Save df into data folder."""    
    # df['Date'] = df.index
    # df = df.set_index("Date")    
    df.to_csv(os.path.join(data_path, filename + ".csv"), index=False)

In [7]:
# grab the datasources we have access to
# pyodbc.dataSources()

In [8]:
# define components of our connection string
driver = '{Microsoft Access Driver (*.mdb, *.accdb)}'
# filepath = r'C:\Users\Alex\OneDrive\Career - Work Items\Petco\Financial_Analyst\Petco_Financial_Data.accdb'
filepath = r'P:\ADMIN\PIIRA\NewPIIRA2004.mdb'

In [9]:
# create a connection to the database
conn = pyodbc.connect(driver = driver, dbq = filepath, autocommit = True)
cursor = conn.cursor()

In [10]:
# grab all the tables
tables_list = list(cursor.tables())
# tables_list

In [11]:
tablas = [list(row)[2] for row in tables_list if list(row)[-2]=='TABLE']
[row for row in tablas if row.find('800')>-1]

['tbl_PIIRAE800', 'tbl_PIIRAE800_bckup111130']

In [12]:
def table_to_rawdata(table_name):
    """Save table into rawdata folder."""    
    
    # define query
    query = "SELECT * FROM {}".format(table_name)
    
    # execute the query
    cursor.execute(query)
    
    df = pd.read_sql(query, conn)
    df_to_rawdata(df, table_name)

In [13]:
table_name = 'tbl_PIIRAWM08'
table_to_rawdata(table_name)

In [14]:
table_name = 'tbl_PIIRAE810_NEW'
table_to_rawdata(table_name)

In [15]:
table_name = 'tbl_PIIRA_CECM810'
table_to_rawdata(table_name)

In [16]:
table_name = 'tbl_PIIRA782B'
table_to_rawdata(table_name)

In [17]:
table_name = 'tbl_PIIRAE800'
table_to_rawdata(table_name)

# 800

In [18]:
filename = "tbl_PIIRAE800"
df = rawdata_to_df(filename)

# M1322

In [19]:
from python_calamine import CalamineWorkbook # !pip install python-calamine

In [20]:
workbook = CalamineWorkbook.from_path("M1322 Data (May 2024 - Oct 2024).xlsx")
vars = workbook.sheet_names

In [21]:
def get_rid():
    var = 'Refinery ID'
    form_list = workbook.get_sheet_by_name(var).to_python()
    df = pd.DataFrame(data=form_list[1:], columns=form_list[0])
    df['RID'] = df['RID'].astype(int)
    df['RID'] = df['RID'].astype(str)
    return df

rid_df = get_rid()

In [22]:
def get_sales2024():

    workbook = CalamineWorkbook.from_path("M1322 Data (May 2024 - Oct 2024).xlsx")
    vars = workbook.sheet_names
    
    var = 'Gasoline Sales'
    form_list = workbook.get_sheet_by_name(var).to_python()
    df = pd.DataFrame(data=form_list[1:], columns=form_list[0])
    df = df[['YEAR', 'MONTH', 'RID', 'WHOLESALE_CHANNEL', 'SALES_VOLUME']]

    # Dealer Tank Wagon (DTW) Sales
    dtw_name_list = ['Dealer Tank Wagon (DTW) Sales', 'Dealer Tankwagon Sales']
    df['WHOLESALE_CHANNEL'] = df['WHOLESALE_CHANNEL'].apply(lambda x: 'DTW Sales' if x in dtw_name_list else x)
    
    df['YEAR'] = df['YEAR'].astype(int)
    df['MONTH'] = df['MONTH'].astype(int)

    df['RID'] = df['RID'].astype(int)
    df['RID'] = df['RID'].astype(str)

    df['SALES_VOLUME'] = pd.to_numeric(df['SALES_VOLUME'], errors='coerce')

    return df

In [23]:
sales2024_df = get_sales2024()

In [24]:
def get_sales2013():

    workbook = CalamineWorkbook.from_path("M1322 Data (2013- April 2024).xlsx")
    vars = workbook.sheet_names
    
    var = 'Gasoline Sales'
    form_list = workbook.get_sheet_by_name(var).to_python()
    df = pd.DataFrame(data=form_list[1:], columns=form_list[0])
    
    # df = df[['YEAR', 'MONTH', 'RID', 'WHOLESALE_CHANNEL', 'SALES_VOLUME']]
    df = df[['YEAR', 'MONTH', 'RID', 'WHOLESALE_GASOLINE', 'VOLUME']]

    df.rename(columns={'WHOLESALE_GASOLINE': 'WHOLESALE_CHANNEL', 'VOLUME': 'SALES_VOLUME'}, inplace=True)

    df['WHOLESALE_CHANNEL'] = df['WHOLESALE_CHANNEL'].map(lambda x: x.title())

    # Dealer Tankwagon Sales
    dtw_name_list = ['Dealer Tank Wagon (DTW) Sales', 'Dealer Tankwagon Sales']
    df['WHOLESALE_CHANNEL'] = df['WHOLESALE_CHANNEL'].apply(lambda x: 'DTW Sales' if x in dtw_name_list else x)
    
    df['YEAR'] = df['YEAR'].astype(int)
    df['MONTH'] = df['MONTH'].astype(int)

    df['RID'] = df['RID'].astype(int)
    df['RID'] = df['RID'].astype(str)

    df['SALES_VOLUME'] = pd.to_numeric(df['SALES_VOLUME'], errors='coerce')

    return df

In [25]:
sales2013_df = get_sales2013()

In [26]:
sales2024_df = get_sales2024()
sales2013_df = get_sales2013()
sales_df = pd.concat([sales2013_df, sales2024_df], ignore_index=True)

In [27]:
sales_df['WHOLESALE_CHANNEL'].unique()

array(['Unbranded Rack Sales', 'Branded Rack Sales', 'Bulk Sales',
       'Spot Pipeline Sales', 'DTW Sales', 'Internally Priced Sales',
       'Sales to Other End-Users', 'Total Gasoline Sales',
       'Company Owned & Operated (COO) Sales'], dtype=object)

In [28]:
df = sales_df[['YEAR', 'MONTH', 'WHOLESALE_CHANNEL', 'SALES_VOLUME']].groupby(['YEAR', 'MONTH', 'WHOLESALE_CHANNEL']).sum()
# df['SALES_VOLUME'] = df['SALES_VOLUME'] / 1_000
df = df.unstack(level='WHOLESALE_CHANNEL')

df.columns.set_names(['Level1', 'WHOLESALE_CHANNEL'], inplace=True)
df.columns = df.columns.droplevel(0)
df = df.reset_index()

df.to_csv('channel.csv', index=False)

# By Refinery

In [29]:
df = sales_df[['YEAR', 'MONTH', 'RID', 'SALES_VOLUME']].groupby(['YEAR', 'MONTH', 'RID']).sum()
# df['SALES_VOLUME'] = df['SALES_VOLUME'] / 1_000
df = df.unstack(level='RID')

df.columns.set_names(['Level1', 'RID'], inplace=True)
df.columns = df.columns.droplevel(0)
df = df.reset_index()

df.to_csv('refinery.csv', index=False)

# Refineries

In [30]:
rid_dict = {}
for index, row in rid_df.iterrows():
    rid_dict[row['RID']] = {
        'ref': row['REFINERY'],
        'op': row['OPERATOR']
    }

In [31]:
for key, value in rid_dict.items():
    RID = key

    df = sales_df[sales_df['RID']==RID]
    
    df = df[['YEAR', 'MONTH', 'WHOLESALE_CHANNEL', 'SALES_VOLUME']]
    
    df.set_index(['YEAR', 'MONTH', 'WHOLESALE_CHANNEL'], inplace=True)
    df = df.unstack(level='WHOLESALE_CHANNEL')
    
    df.columns.set_names(['Level1', 'WHOLESALE_CHANNEL'], inplace=True)
    df.columns = df.columns.droplevel(0)
    df = df.reset_index()
    
    df.to_csv(RID + '.csv', index=False)


# 782

In [32]:
filename = "tbl_PIIRA782B"
df = rawdata_to_df(filename)

In [33]:
sorted(df['Year'].unique())

[2006, 2007, 2008]

# 08

In [34]:
filename = "tbl_PIIRAWM08"
df = rawdata_to_df(filename)

# 810

In [35]:
filename = "tbl_PIIRAE810_NEW"
df = rawdata_to_df(filename)

In [36]:
filename = "tbl_PIIRA_CECM810"
df = rawdata_to_df(filename)

In [37]:
filename = "tbl_PIIRAE810_NEW"
df = rawdata_to_df(filename)
# df['CID'] = df['CID'].astype(int)
# df['CID'] = df['CID'].astype(str)

# Product_Code: Description (Product_Code.csv)
df_product_code = pd.read_csv("Product_Code.csv")
df_product_code = df_product_code[:294]
df_product_code['Product_Code'] = df_product_code['Product_Code'].astype(int) 
df_product_code.dtypes

dict_product_code = {}
for code in df_product_code['Product_Code']:
    dict_product_code[int(code)] = df_product_code[df_product_code['Product_Code']==code]['Description'].values[0]

df['Description'] = df['PRODUCT_CODE'].map(dict_product_code)

filename = "tbl_PIIRA_CECM810"
df = rawdata_to_df(filename)
# df['CID'] = df['CID'].astype(int)
# df['CID'] = df['CID'].astype(str)

# Product_Code: Description (Product_Code.csv)
df_product_code = pd.read_csv("Product_Code.csv")
df_product_code = df_product_code[:294]
df_product_code['Product_Code'] = df_product_code['Product_Code'].astype(int) 
df_product_code.dtypes

dict_product_code = {}
for code in df_product_code['Product_Code']:
    dict_product_code[int(code)] = df_product_code[df_product_code['Product_Code']==code]['Description'].values[0]

df['Description'] = df['PRODUCT_CODE'].map(dict_product_code)

df = df[["YEAR", "MONTH", "RID", "PRODUCT_CODE", "ENTRY_FIELD", "VALUE"]]

df.loc[df.ENTRY_FIELD=="a", 'ENTRY_FIELD'] = "A"
df.loc[df.ENTRY_FIELD=="c", 'ENTRY_FIELD'] = "C"
df.loc[df.ENTRY_FIELD=="d", 'ENTRY_FIELD'] = "D"
df.loc[df.ENTRY_FIELD=="e", 'ENTRY_FIELD'] = "E"
df.loc[df.ENTRY_FIELD=="g", 'ENTRY_FIELD'] = "G"

df = df.pivot(index=["YEAR", "MONTH", "RID", "PRODUCT_CODE"], columns='ENTRY_FIELD', values='VALUE')
df = df.reset_index()

# Product_Code: Description (Product_Code.csv)
df_product_code = pd.read_csv("Product_Code.csv")
df_product_code = df_product_code[:294]
df_product_code['Product_Code'] = df_product_code['Product_Code'].astype(int) 

dict_product_code = {}
for code in df_product_code['Product_Code']:
    dict_product_code[int(code)] = df_product_code[df_product_code['Product_Code']==code]['Description'].values[0]

df['Labels PRODUCT_CODE'] = df['PRODUCT_CODE'].map(dict_product_code)

# df.to_csv("CECM810.csv", index=False)
df_to_data(df, "CECM810.csv")



filename = "tbl_PIIRA_CECM810"
df = rawdata_to_df(filename)
# df['CID'] = df['CID'].astype(int)
# df['CID'] = df['CID'].astype(str)

# Product_Code: Description (Product_Code.csv)
df_product_code = pd.read_csv("Product_Code.csv")
df_product_code = df_product_code[:294]
df_product_code['Product_Code'] = df_product_code['Product_Code'].astype(int) 
df_product_code.dtypes

dict_product_code = {}
for code in df_product_code['Product_Code']:
    dict_product_code[int(code)] = df_product_code[df_product_code['Product_Code']==code]['Description'].values[0]

df['Description'] = df['PRODUCT_CODE'].map(dict_product_code)

df['RID'].unique()

dict_rid = {
    75005:	'Marathon Carson Wilmington',
    519010:	'PBF Torrance',
    533001:	'Kern Bakersfield',
    749007:	'PBF Martinez',
    810103:	'Chevron El Segundo',
    810104:	'Chevron Richmond',
    816106:	'Valero Benicia',
    897101:	'Valero Wilmington',
    871203:	'P66 Rodeo',
    871205:	'P66 Wilmington'
}

def get_df_rid(RID):
    y = df[df['RID']==RID].copy()
    return y        

dict_entry_field = {
    'A': 'Opening Inventory',
    'B': 'Receipts (Imports)',
    'C': 'Inputs',
    'D': 'Production',
    'E': 'Shipments (Exports)',
    'F': 'Fuel Use (Energy)',
    'G': 'Closing Inventory'   
}

# Production
# ENTRY_FIELD: D

# PRODUCT_CODE:
# Reformulated Blendstock for Oxygenate Blending (RBOB): 118
# Unfinished Oils, Total: 812
# Heavy Gas Oils: 840
# Distillate Fuel Oil, Total: 411
# Distillate Fuel Oil, 15ppm Sulfur and Under: 465
# Renewable Fuels, Total: 202
# Other Renewable Diesel Fuel (R100): 205
# Conventional Blendstock for Oxygenate Blending (CBOB): 139
# Petroleum Coke, Marketable: 21
# Still Gas: 45
# GTAB All Other Mogas Blending Components: 138
# Kerosene and Light Gas Oils: 830
# Other Renewable Fuels: 207

df_D = df[df['ENTRY_FIELD']=='D'].copy()

product_list = [118, 812, 840, 411, 465, 202, 205, 139, 21, 45, 138, 830, 207]

df_D = df_D[df_D['PRODUCT_CODE'].isin(product_list)]

# Inputs
# ENTRY_FIELD: C

# Crude Oil (incl. lease condensate), TOTAL	50
# Renewable Fuels, Total	202
# Other Renewable Diesel Fuel (R100)	205
# GTAB All Other Mogas Blending Components	138
# Inputs (Gain) or Production (Loss)	911
# Unfinished Oils, Total	812
# Naphthas and Lighter	820
# Residuum	850
# Distillate Fuel Oil, Total	411
# Distillate Fuel Oil, 15ppm Sulfur and Under	465
# Kerosene and Light Gas Oils	830
# Heavy Gas Oils	840
# Normal Butane/Butylene - Total	244
# Normal Butane -LRG	643

df_C = df[df['ENTRY_FIELD']=='C'].copy()

product_list = [55, 202, 205, 138, 911, 812, 820, 850, 411, 465, 830, 840, 244, 643]

df_C = df_C[df_C['PRODUCT_CODE'].isin(product_list)]

# Energy
# ENTRY_FIELD: F

# Still Gas	45
# Petroleum Coke, Catalyst	22

df_F = df[df['ENTRY_FIELD']=='F'].copy()

product_list = [45, 22]

df_F = df_F[df_F['PRODUCT_CODE'].isin(product_list)]

df = pd.concat([df_D,df_C,df_F], axis='rows')
df['VAR'] = df['ENTRY_FIELD'].astype(str) + df['PRODUCT_CODE'].astype(str)
df = df[['YEAR', 'MONTH', 'RID', 'VAR', 'VALUE']]
df = df.pivot(index=['YEAR', 'MONTH', 'RID'], columns='VAR', values='VALUE') 
df = df.reset_index()

# df.to_csv("E810.csv", index=False)
df_to_data(df, "E810.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Product_Code.csv'